# Take some RNN models from tensorflow seq2seq:

TODO:

- [x] For test data, show input batch words, output words, and predicted words (and print this to a file every so often)

- [x] Switch to `embedding_rnn_seq2seq` model

- [x] Capture word embeddings

- Normalize word embeddings

- [x] Implement VAE

- Capture z's and latent loss, and minimize latent loss

- [x] Get good summaries of Error, etc written to files during training

- Any more finishing of model using original en -> fr data. Such as saving the tensorboard graph model.

Figures:

1. model diagram
2. Training curves
3. token embeddings
4. visualize latent (color by sequence families or GO category)
5. reconstruction (multiple seq alignment)

## Now simply apply to protein data

- Run protein data through the system using same tokenization machinery as for en -> fr translation

- Have to change bucket sizes appropriatly

- Tokenize 2 amino acids to get 400 total combos because easier to analyze embeddings. See if order matters ie does XY = YX ?

- Color latent space by sequence attributes

## (Maybe) do the same thing, but for protein structure 

- (going to require more work since no longer dealing with discrete tokens)

- Have to encode real valued positions, orientations etc .......

# VAE model

In [1]:
import numpy as np
import tensorflow as tf

def make_vae(x):
  """ Takes a numeric tensor and returns the reconstructed mean tensor of same dims
  (batch_size, input_dim)"""

  # A lot of this code section is taken and modified from https://jmetzen.github.io/2015-11-27/vae.html

  # hyper params:
  n_hidden_recog_1=500 # 1st layer encoder neurons
  n_hidden_recog_2=500 # 2nd layer encoder neurons

  n_hidden_gener_1=500 # 1st layer decoder neurons
  n_hidden_gener_2=500 # 2nd layer decoder neurons
  n_input=size*2
  n_z=20

  # CREATE NETWORK

  # 1) input placeholder

  # 2) weights and biases variables
  def xavier_init(fan_in, fan_out, constant=1): 
      """ Xavier initialization of network weights"""
      # https://stackoverflow.com/questions/33640581/how-to-do-xavier-initialization-on-tensorflow
      low = -constant*np.sqrt(6.0/(fan_in + fan_out)) 
      high = constant*np.sqrt(6.0/(fan_in + fan_out))
      return tf.random_uniform((fan_in, fan_out), minval=low, maxval=high, dtype=tf.float32)

  wr_h1 = tf.Variable(xavier_init(n_input, n_hidden_recog_1))
  wr_h2 = tf.Variable(xavier_init(n_hidden_recog_1, n_hidden_recog_2))
  wr_out_mean = tf.Variable(xavier_init(n_hidden_recog_2, n_z))
  wr_out_log_sigma = tf.Variable(xavier_init(n_hidden_recog_2, n_z))

  br_b1 = tf.Variable(tf.zeros([n_hidden_recog_1], dtype=tf.float32))
  br_b2 = tf.Variable(tf.zeros([n_hidden_recog_2], dtype=tf.float32))
  br_out_mean = tf.Variable(tf.zeros([n_z], dtype=tf.float32))
  br_out_log_sigma = tf.Variable(tf.zeros([n_z], dtype=tf.float32))

  wg_h1 = tf.Variable(xavier_init(n_z, n_hidden_gener_1))
  wg_h2 = tf.Variable(xavier_init(n_hidden_gener_1, n_hidden_gener_2))
  wg_out_mean = tf.Variable(xavier_init(n_hidden_gener_2, n_input))
  # wg_out_log_sigma = tf.Variable(xavier_init(n_hidden_gener_2, n_input))

  bg_b1 = tf.Variable(tf.zeros([n_hidden_gener_1], dtype=tf.float32))
  bg_b2 = tf.Variable(tf.zeros([n_hidden_gener_2], dtype=tf.float32))
  bg_out_mean = tf.Variable(tf.zeros([n_input], dtype=tf.float32))

  # 3) recognition network

  # use recognition network to predict mean and (log) variance of (latent) Gaussian distribution z (n_z dimensional)
  r_layer_1 = tf.nn.softplus(tf.add(tf.matmul(x, wr_h1), br_b1))
  r_layer_2 = tf.nn.softplus(tf.add(tf.matmul(r_layer_1, wr_h2), br_b2))
  z_mean = tf.add(tf.matmul(r_layer_2, wr_out_mean), br_out_mean)
  z_sigma = tf.add(tf.matmul(r_layer_2, wr_out_log_sigma), br_out_log_sigma)

  # 4) do sampling on recognition network to get latent variables
  # draw one n_z dimensional sample (for each input in batch), from normal distribution
  eps = tf.random_normal((batch_size, n_z), 0, 1, dtype=tf.float32)

  # scale that set of samples by predicted mu and epsilon to get samples of z, the latent distribution
  # z = mu + sigma*epsilon
  latent_z = tf.add(z_mean, tf.mul(tf.sqrt(tf.exp(z_sigma)), eps), name="latent_z")

  # 5) use generator network to predict mean of Bernoulli distribution of reconstructed input
  g_layer_1 = tf.nn.softplus(tf.add(tf.matmul(latent_z, wg_h1), bg_b1))
  g_layer_2 = tf.nn.softplus(tf.add(tf.matmul(g_layer_1, wg_h2), bg_b2))
  x_reconstr_mean = tf.nn.sigmoid(tf.add(tf.matmul(g_layer_2, wg_out_mean), bg_out_mean))

  # 6) loss
  latent_loss = tf.mul(-0.5, tf.reduce_sum(1 + z_sigma - tf.square(z_mean) - tf.exp(z_sigma), 1), name="latent_loss")

  return x_reconstr_mean, latent_z, latent_loss

# model components from tensorflow

In [2]:
from tensorflow.python.framework import dtypes
from tensorflow.python.framework import ops
from tensorflow.python.ops import rnn
from tensorflow.python.ops import rnn_cell
from tensorflow.python.ops import variable_scope
from tensorflow.python.util import nest

from seq2seq import sequence_loss, embedding_rnn_decoder

def model_with_buckets(encoder_inputs, decoder_inputs, targets, weights,
                       buckets, seq2seq, softmax_loss_function=None, name=None):
  """taken (with modification from tensorflow) so look up docstrings elsewhere"""
  if len(encoder_inputs) < buckets[-1][0]:
    raise ValueError("Length of encoder_inputs (%d) must be at least that of la"
                     "st bucket (%d)." % (len(encoder_inputs), buckets[-1][0]))
  if len(targets) < buckets[-1][1]:
    raise ValueError("Length of targets (%d) must be at least that of last"
                     "bucket (%d)." % (len(targets), buckets[-1][1]))
  if len(weights) < buckets[-1][1]:
    raise ValueError("Length of weights (%d) must be at least that of last"
                     "bucket (%d)." % (len(weights), buckets[-1][1]))

  all_inputs = encoder_inputs + decoder_inputs + targets + weights
  losses = []
  outputs = []
  latent_losses = []
  latent_zs = []
  with ops.op_scope(all_inputs, name, "model_with_buckets"):
    for j, bucket in enumerate(buckets):
      with variable_scope.variable_scope(variable_scope.get_variable_scope(),
                                         reuse=True if j > 0 else None):
        bucket_outputs, _ = seq2seq(encoder_inputs[:bucket[0]],
                                    decoder_inputs[:bucket[1]])
        outputs.append(bucket_outputs)
#         latent_losses.append(latent_loss)
#         latent_zs.append(latent_z)
        
        losses.append(sequence_loss(
            outputs[-1], targets[:bucket[1]], weights[:bucket[1]],
            softmax_loss_function=softmax_loss_function))

  return outputs, losses


def embedding_rnn_seq2seq(encoder_inputs, decoder_inputs, cell,
                          num_encoder_symbols, num_decoder_symbols,
                          embedding_size, output_projection=None,
                          feed_previous=False, dtype=dtypes.float32,
                          scope=None,
                         ):
  """taken (with modification from tensorflow) so look up docstrings elsewhere"""
  with variable_scope.variable_scope(scope or "embedding_rnn_seq2seq"):
    # Encoder.
    encoder_cell = rnn_cell.EmbeddingWrapper(
        cell, embedding_classes=num_encoder_symbols,
        embedding_size=embedding_size)
    _, encoder_state = rnn.rnn(encoder_cell, encoder_inputs, dtype=dtype)
    
    # Patrick: keep your eyes on that encoder state
    encoder_state, latent_z, latent_loss = make_vae(encoder_state)

    # Decoder.
    if output_projection is None:
      cell = rnn_cell.OutputProjectionWrapper(cell, num_decoder_symbols)

    if isinstance(feed_previous, bool):
      return embedding_rnn_decoder(
          decoder_inputs, encoder_state, cell, num_decoder_symbols,
          embedding_size, output_projection=output_projection,
          feed_previous=feed_previous)

    # If feed_previous is a Tensor, we construct 2 graphs and use cond.
    def decoder(feed_previous_bool):
      reuse = None if feed_previous_bool else True
      with variable_scope.variable_scope(variable_scope.get_variable_scope(),
                                         reuse=reuse):
        outputs, state = embedding_rnn_decoder(
            decoder_inputs, encoder_state, cell, num_decoder_symbols,
            embedding_size, output_projection=output_projection,
            feed_previous=feed_previous_bool,
            update_embedding_for_previous=False)
        state_list = [state]
        if nest.is_sequence(state):
          state_list = nest.flatten(state)
        return outputs + state_list

    outputs_and_state = control_flow_ops.cond(feed_previous,
                                              lambda: decoder(True),
                                              lambda: decoder(False))
    outputs_len = len(decoder_inputs)  # Outputs length same as decoder inputs.
    state_list = outputs_and_state[outputs_len:]
    state = state_list[0]
    if nest.is_sequence(encoder_state):
      state = nest.pack_sequence_as(structure=encoder_state, flat_sequence=state_list)
    
    return outputs_and_state[:outputs_len], state, latent_z, latent_loss

# The translation model:

In [3]:
import random

import numpy as np
import tensorflow as tf

import data_utils


class Seq2SeqModel(object):
  """taken (with modification from tensorflow) so look up docstrings elsewhere"""
  def __init__(self,
               source_vocab_size,
               target_vocab_size,
               buckets,
               size,
               num_layers,
               max_gradient_norm,
               batch_size,
               learning_rate,
               learning_rate_decay_factor,
               use_lstm=False,
               num_samples=512,
               forward_only=False,
               dtype=tf.float32):
    self.source_vocab_size = source_vocab_size
    self.target_vocab_size = target_vocab_size
    self.buckets = buckets
    self.batch_size = batch_size
    self.learning_rate = tf.Variable(
        float(learning_rate), trainable=False, dtype=dtype)
    self.learning_rate_decay_op = self.learning_rate.assign(
        self.learning_rate * learning_rate_decay_factor)
    self.global_step = tf.Variable(0, trainable=False)

    # If we use sampled softmax, we need an output projection.
    output_projection = None
    softmax_loss_function = None
    # Sampled softmax only makes sense if we sample less than vocabulary size.
    if num_samples > 0 and num_samples < self.target_vocab_size:
      w_t = tf.get_variable("proj_w", [self.target_vocab_size, size], dtype=dtype)
      w = tf.transpose(w_t)
      b = tf.get_variable("proj_b", [self.target_vocab_size], dtype=dtype)
      output_projection = (w, b)

      def sampled_loss(inputs, labels):
        labels = tf.reshape(labels, [-1, 1])
        # We need to compute the sampled_softmax_loss using 32bit floats to
        # avoid numerical instabilities.
        local_w_t = tf.cast(w_t, tf.float32)
        local_b = tf.cast(b, tf.float32)
        local_inputs = tf.cast(inputs, tf.float32)
        return tf.cast(
            tf.nn.sampled_softmax_loss(local_w_t, local_b, local_inputs, labels,
                                       num_samples, self.target_vocab_size),dtype)
      
      softmax_loss_function = sampled_loss

    # Create the internal multi-layer cell for our RNN.
    single_cell = tf.nn.rnn_cell.GRUCell(size)
    if use_lstm:
      single_cell = tf.nn.rnn_cell.BasicLSTMCell(size)
    cell = single_cell
    if num_layers > 1:
      cell = tf.nn.rnn_cell.MultiRNNCell([single_cell] * num_layers)

    # The seq2seq function: we use embedding for the input and attention.
    def seq2seq_f(encoder_inputs, decoder_inputs, do_decode):
      return embedding_rnn_seq2seq(
          encoder_inputs,
          decoder_inputs,
          cell,
          num_encoder_symbols=source_vocab_size,
          num_decoder_symbols=target_vocab_size,
          embedding_size=size,
          output_projection=output_projection,
          feed_previous=do_decode,
          dtype=dtype)

    # Feeds for inputs.
    self.encoder_inputs = []
    self.decoder_inputs = []
    self.target_weights = []
    for i in range(buckets[-1][0]):  # Last bucket is the biggest one.
      self.encoder_inputs.append(tf.placeholder(tf.int32, shape=[None],
                                                name="encoder{0}".format(i)))
    for i in range(buckets[-1][1] + 1):
      self.decoder_inputs.append(tf.placeholder(tf.int32, shape=[None],
                                                name="decoder{0}".format(i)))
      self.target_weights.append(tf.placeholder(dtype, shape=[None],
                                                name="weight{0}".format(i)))

    # Our targets are decoder inputs shifted by one.
    targets = [self.decoder_inputs[i + 1]
               for i in range(len(self.decoder_inputs) - 1)]
    
    # Training outputs and losses.
    if forward_only:
      self.outputs, self.losses = model_with_buckets(
          self.encoder_inputs, self.decoder_inputs, targets,
          self.target_weights, buckets, lambda x, y: seq2seq_f(x, y, True),
          softmax_loss_function=softmax_loss_function)
      # If we use output projection, we need to project outputs for decoding.
      if output_projection is not None:
        for b in range(len(buckets)):
          self.outputs[b] = [
              tf.matmul(output, output_projection[0]) + output_projection[1]
              for output in self.outputs[b]
          ]
    else:
      self.outputs, self.losses = model_with_buckets(
          self.encoder_inputs, self.decoder_inputs, targets,
          self.target_weights, buckets,
          lambda x, y: seq2seq_f(x, y, False),
          softmax_loss_function=softmax_loss_function)

    # Gradients and SGD update operation for training the model.
    params = tf.trainable_variables()
    if not forward_only:
      self.gradient_norms = []
      self.updates = []
      opt = tf.train.GradientDescentOptimizer(self.learning_rate)
      for b in range(len(buckets)):
        gradients = tf.gradients(self.losses[b], params)
        clipped_gradients, norm = tf.clip_by_global_norm(gradients,
                                                         max_gradient_norm)
        self.gradient_norms.append(norm)
        self.updates.append(opt.apply_gradients(
            zip(clipped_gradients, params), global_step=self.global_step))

    self.saver = tf.train.Saver(tf.all_variables())

  def step(self, session, encoder_inputs, decoder_inputs, target_weights,
           bucket_id, forward_only):

    # encoder and decoder sizes must match the corresponding bucket

    # Input feed: encoder inputs, decoder inputs, target_weights, as provided.
    input_feed = {}
    for l in range(encoder_size):
      input_feed[self.encoder_inputs[l].name] = encoder_inputs[l]
    for l in range(decoder_size):
      input_feed[self.decoder_inputs[l].name] = decoder_inputs[l]
      input_feed[self.target_weights[l].name] = target_weights[l]

    # Since our targets are decoder inputs shifted by one, we need one more.
    last_target = self.decoder_inputs[decoder_size].name
    input_feed[last_target] = np.zeros([self.batch_size], dtype=np.int32)

    # Output feed: depends on whether we do a backward step or not.
    if not forward_only:
      output_feed = [self.updates[bucket_id],  # Update Op that does SGD.
                     self.gradient_norms[bucket_id],  # Gradient norm.
                     self.losses[bucket_id]
                    ]  
    else:
      output_feed = [self.losses[bucket_id]]  # Loss for this batch.
      
      for l in range(decoder_size):  # Output logits.
        output_feed.append(self.outputs[bucket_id][l])

    outputs = session.run(output_feed, input_feed)
    if not forward_only:
      return outputs[1], outputs[2], None  # Gradient norm, loss, no outputs.
    else:
      return None, outputs[0], outputs[1:-2]   # No gradient norm, loss, outputs.

    
  def get_batch(self, data, bucket_id):

    encoder_size, decoder_size = self.buckets[bucket_id]
    encoder_inputs, decoder_inputs = [], []

    # Get a random batch of encoder and decoder inputs from data,
    # pad them if needed, reverse encoder inputs and add GO to decoder.
    for _ in range(self.batch_size):
      encoder_input, decoder_input = random.choice(data[bucket_id])

      # Encoder inputs are padded and then reversed.
      encoder_pad = [data_utils.PAD_ID] * (encoder_size - len(encoder_input))
      encoder_inputs.append(list(reversed(encoder_input + encoder_pad)))

      # Decoder inputs get an extra "GO" symbol, and are padded then.
      decoder_pad_size = decoder_size - len(decoder_input) - 1
      decoder_inputs.append([data_utils.GO_ID] + decoder_input +
                            [data_utils.PAD_ID] * decoder_pad_size)

    # Now we create batch-major vectors from the data selected above.
    batch_encoder_inputs, batch_decoder_inputs, batch_weights = [], [], []

    # Batch encoder inputs are just re-indexed encoder_inputs.
    for length_idx in range(encoder_size):
      batch_encoder_inputs.append(
          np.array([encoder_inputs[batch_idx][length_idx]
                    for batch_idx in range(self.batch_size)], dtype=np.int32))

    # Batch decoder inputs are re-indexed decoder_inputs, we create weights.
    for length_idx in range(decoder_size):
      batch_decoder_inputs.append(
          np.array([decoder_inputs[batch_idx][length_idx]
                    for batch_idx in range(self.batch_size)], dtype=np.int32))

      # Create target_weights to be 0 for targets that are padding.
      batch_weight = np.ones(self.batch_size, dtype=np.float32)
      for batch_idx in range(self.batch_size):
        # We set weight to 0 if the corresponding target is a PAD symbol.
        # The corresponding target is decoder_input shifted by 1 forward.
        if length_idx < decoder_size - 1:
          target = decoder_inputs[batch_idx][length_idx + 1]
        if length_idx == decoder_size - 1 or target == data_utils.PAD_ID:
          batch_weight[batch_idx] = 0.0
      batch_weights.append(batch_weight)
    return batch_encoder_inputs, batch_decoder_inputs, batch_weights
  
def make_dir(directory):
  if not os.path.exists(directory):
    os.makedirs(directory)
  return directory

# Do training:

In [4]:
import math
import os
import random
import sys
import time

import numpy as np
import pandas as pd
import tensorflow as tf

import data_utils

learning_rate = 0.5
learning_rate_decay_factor = 0.99
max_gradient_norm = 5.0
batch_size = 64
size = 512 # both the embedding size and the RNN state size
num_layers = 2
en_vocab_size = 40000
fr_vocab_size = 40000
data_dir = make_dir("seq2seq_data/")
train_dir = make_dir("seq2seq_train/")
max_train_data_size = 100000
steps_per_checkpoint = 2000
do_decode = False
do_self_test = False

# We use a number of buckets and pad to the closest one for efficiency.
# See Seq2SeqModel for details of how they work.
_buckets = [(5, 10), (10, 15), (20, 25), (40, 50)]

def read_data(source_path, target_path, max_size=None):
  """taken (with modification from tensorflow) so look up docstrings elsewhere"""
  data_set = [[] for _ in _buckets]
  with tf.gfile.GFile(source_path, mode="r") as source_file:
    with tf.gfile.GFile(target_path, mode="r") as target_file:
      source, target = source_file.readline(), target_file.readline()
      counter = 0
      while source and target and (not max_size or counter < max_size):
        counter += 1
        if counter % 100000 == 0:
          print("  reading data line %d" % counter)
          sys.stdout.flush()
        source_ids = [int(x) for x in source.split()]
        target_ids = [int(x) for x in target.split()]
        target_ids.append(data_utils.EOS_ID)
        for bucket_id, (source_size, target_size) in enumerate(_buckets):
          if len(source_ids) < source_size and len(target_ids) < target_size:
            data_set[bucket_id].append([source_ids, target_ids])
            break
        source, target = source_file.readline(), target_file.readline()
  return data_set


def create_model(session, forward_only):
  """Create translation model and initialize or load parameters in session."""
  model = Seq2SeqModel(
      en_vocab_size,
      fr_vocab_size,
      _buckets,
      size,
      num_layers,
      max_gradient_norm,
      batch_size,
      learning_rate,
      learning_rate_decay_factor,
      forward_only=forward_only)

  ckpt = tf.train.get_checkpoint_state(train_dir)
  if ckpt and tf.gfile.Exists(ckpt.model_checkpoint_path):
    print("Reading model parameters from %s" % ckpt.model_checkpoint_path)
    model.saver.restore(session, ckpt.model_checkpoint_path)
  else:
    print("Created model with fresh parameters.")
    session.run(tf.initialize_all_variables())
  return model

def self_test():
  """Test the translation model."""
  with tf.Session() as sess: # make new session (no loading of saved session)
    print("Self-test for neural translation model.")
    # Create model with vocabularies of 10, 2 small buckets, 2 layers of 32.
    model = Seq2SeqModel(
        10, # source vocab size
        10, # target vocab size
        [(3, 3),
         (6, 6)], # buckets (max_en, max_fr)
        1024, # size
        2, # num layers
        5.0, # max gradient norm
        32, # batch size
        0.3, # learning rate
        0.99, # learning rate decay factor
        use_lstm=False,
        num_samples=8, # number of samples for sampled softmax
        forward_only=False, # run 1-directional rnn (or something like that)
        )
    sess.run(tf.initialize_all_variables())

    data_set = ([([1, 1], [2, 2]), ([3, 3], [4]), ([5], [6])],
                [([1, 1, 1, 1, 1], [2, 2, 2, 2, 2]), ([3, 3, 3], [5, 6])])
    for _ in range(10):  # Train the fake model for 5 steps.
      bucket_id = random.choice([0, 1]) # choose a bucket
      encoder_inputs, decoder_inputs, target_weights = model.get_batch(data_set, bucket_id)
      _, step_loss, output_logits = model.step(sess, encoder_inputs, decoder_inputs, target_weights, bucket_id, False)
      print("step_loss: {}".format(step_loss))
      
def write_batch_input_output_prediction_to_file(train_dir, bucket_num, global_step):
  s = ""
  # inputs
  enc_inputs_npa = np.array([list(ar) for ar in encoder_inputs]).T
  for sent_id in range(enc_inputs_npa.shape[0]):
    sent = enc_inputs_npa[sent_id]
    s += "  " + str(sent_id) + ": " + \
         " ".join(reversed([rev_en_vocab[n].decode("utf8") for n in sent if n != 0])) + "\n"

  # outputs
  decoder_inputs_npa = np.array([list(ar) for ar in decoder_inputs]).T
  for sent_id in range(decoder_inputs_npa.shape[0]):
    sent = decoder_inputs_npa[sent_id]
    s += "  " + str(sent_id) + ": " + \
         " ".join([rev_fr_vocab[n].decode("utf8") for n in sent if n != 0]) + "\n"

  # predicted outputs
  pred_outputs = [list(np.argmax(logit, axis=1)) for logit in output_logits]
  pred_outputs = np.array(pred_outputs).T
  for sent_id in range(pred_outputs.shape[0]):
    sent = pred_outputs[sent_id]
    s += "  " + str(sent_id) + ": " + \
         " ".join([rev_fr_vocab[n].decode("utf8") for n in sent if n != 0]) + "\n"

  with open(train_dir + "batch_trans_" + str(int(g_step)) + \
       "_bucket_" + str(bucket_id) + ".txt", "w") as f:
    f.write(s)

if do_self_test:
  self_test()
else:
  sess = tf.Session()
  """Train a en->fr translation model using WMT data."""
  # Prepare WMT data.
  print("Preparing WMT data in %s" % data_dir)
  en_train, fr_train, en_dev, fr_dev, _, _ = data_utils.prepare_wmt_data(
      data_dir, en_vocab_size, fr_vocab_size)

  # Load vocabularies
  en_vocab_path = os.path.join(data_dir, "vocab%d.en" % en_vocab_size)
  fr_vocab_path = os.path.join(data_dir, "vocab%d.fr" % fr_vocab_size)
  en_vocab, rev_en_vocab = data_utils.initialize_vocabulary(en_vocab_path)
  fr_vocab, rev_fr_vocab = data_utils.initialize_vocabulary(fr_vocab_path)

  sess = tf.Session()
  # Create model.
  print("Creating %d layers of %d units." % (num_layers, size))
  model = create_model(sess, False)

  # Read data into buckets and compute their sizes.
  print ("Reading development and training data (limit: %d)."
         % max_train_data_size)
  dev_set = read_data(en_dev, fr_dev)
  train_set = read_data(en_train, fr_train, max_train_data_size)
  train_bucket_sizes = [len(train_set[b]) for b in range(len(_buckets))]
  train_total_size = float(sum(train_bucket_sizes))

  # A bucket scale is a list of increasing numbers from 0 to 1 that we'll use
  # to select a bucket. Length of [scale[i], scale[i+1]] is proportional to
  # the size if i-th training bucket, as used later.
  train_buckets_scale = [sum(train_bucket_sizes[:i + 1]) / train_total_size
                         for i in range(len(train_bucket_sizes))]

  # This is the training loop.
  step_time, loss = 0.0, 0.0
  current_step = 0
  previous_losses = []
  while True:
    # Choose a bucket according to data distribution. We pick a random number
    # in [0, 1] and use the corresponding interval in train_buckets_scale.
    random_number_01 = np.random.random_sample()
    bucket_id = min([i for i in range(len(train_buckets_scale))
                     if train_buckets_scale[i] > random_number_01])

    # Get a batch and make a step.
    start_time = time.time()
    encoder_inputs, decoder_inputs, target_weights = model.get_batch(
        train_set, bucket_id)
    _, step_loss, _ = model.step(sess, encoder_inputs, decoder_inputs,
                                 target_weights, bucket_id, False)
    step_time += (time.time() - start_time) / steps_per_checkpoint
    loss += step_loss / steps_per_checkpoint
    current_step += 1

    # Once in a while, we save checkpoint, print statistics, and run evals.
    if True: # current_step % steps_per_checkpoint == 0: # --------------------------------------
      print("logging summaries and errors in: " + train_dir)
      # Print statistics for the previous epoch.
      perplexity = math.exp(float(loss)) if loss < 300 else float("inf")
      g_step = sess.run(model.global_step)
      l_rate = sess.run(model.learning_rate)
      with open(train_dir + "error_train_log.txt", "a") as f:
        f.write("train {} {} {} {}\n".format(g_step, l_rate, step_time, perplexity))
      # Decrease learning rate if no improvement was seen over last 3 times.
      if len(previous_losses) > 2 and loss > max(previous_losses[-3:]):
        sess.run(model.learning_rate_decay_op)
      previous_losses.append(loss)
      # Save checkpoint and zero timer and loss.
      checkpoint_path = os.path.join(train_dir, "translate.ckpt")
#       model.saver.save(sess, checkpoint_path, global_step=model.global_step)
      step_time, loss = 0.0, 0.0
      # Run evals on development set and print their perplexity. --------------------------
      for bucket_id in range(len(_buckets)):
        if len(dev_set[bucket_id]) == 0:
          with open(train_dir + "error_log.txt", "a") as f:
            f.write("test {} empty\n".format(bucket_id))
          continue
        encoder_inputs, decoder_inputs, target_weights = model.get_batch(dev_set, bucket_id)
        _, eval_loss, output_logits = model.step(sess, encoder_inputs,
                                                 decoder_inputs, target_weights, bucket_id, True)
        eval_ppx = math.exp(float(eval_loss)) if eval_loss < 300 else float("inf")
        with open(train_dir + "error_test_log.txt", "a") as f:
          f.write("test {} {}\n".format(bucket_id, eval_ppx))
        
        # print log of latent space (over the test batch)
#         with open(train_dir + "latent_zs.txt", "a") as f:
#           f.write("{} {}\n".format(bucket_id, lat_z))
          
        # print text example translations
        write_batch_input_output_prediction_to_file(train_dir, bucket_id, g_step)
        
      # print out encoder embeddings as pandas dframe --------------------------------------
      for var in tf.trainable_variables():
        if var.name == 'embedding_rnn_seq2seq/RNN/EmbeddingWrapper/embedding:0':
            input_embedd_op = var
      input_embedd_ar = sess.run(input_embedd_op)
      df = pd.DataFrame(data=input_embedd_ar, index=[
                rev_en_vocab[i].decode("utf8") for i in range(input_embedd_ar.shape[0])])
      df.head().to_csv(train_dir + "embedd_df_" + str(int(g_step)) + ".csv")
      # df.to_csv(train_dir + "embedd_df_" + str(int(g_step)) + ".csv")
      
      # -------------------------------------------------------------------------------------

      # break to capture environment for debug
      break


Preparing WMT data in seq2seq_data/
Creating 2 layers of 512 units.
Created model with fresh parameters.
Reading development and training data (limit: 100000).
  reading data line 100000
logging summaries and errors in: seq2seq_train/
